In [43]:
import pulp as pl
import numpy as np
# from scipy.stats import multivariate_t

print("==============================================================================")

# Define the problem
model = pl.LpProblem("Maximize_Profit", pl.LpMaximize)

# Products, machines and days
subproducts = ['Ciasto_na_chleb_domowy_0_5kg', 'Ciasto_na_chleb_domowy_0_9kg', 'Ciasto_na_chleb_na_zakwasie', 'Ciasto_na_bulki', 'Ciasto_na_bagietki', 'Ciasto_na_rogaliki_z_czekolada']
products = ['Chleb_domowy_0_5kg', 'Chleb_domowy_0_9kg', 'Chleb_na_zakwasie', 'Bulki', 'Bagietki', 'Rogaliki_z_czekolada']
days = ['Pon', 'Wt', 'Sr']
sub_machines = ['Dzieza', 'Wazenie', 'Dzielarka', 'Tabula', 'Rogalikarka', 'Nadziewarka']
machines = ['Garownia', 'Piec', 'Studzenie', 'Pakowanie']

# Income from product sales and production cost in PLN/piece
product_income = {'Chleb_domowy_0_5kg': 5.2, 'Chleb_domowy_0_9kg': 7.9, 'Chleb_na_zakwasie': 6.2, 'Bulki': 1.4, 'Bagietki': 1.55, 'Rogaliki_z_czekolada': 2.1}
sub_product_cost = {'Ciasto_na_chleb_domowy_0_5kg': 1.6, 'Ciasto_na_chleb_domowy_0_9kg': 2.0, 'Ciasto_na_chleb_na_zakwasie': 2.0, 'Ciasto_na_bulki': 0.5, 'Ciasto_na_bagietki': 0.6, 'Ciasto_na_rogaliki_z_czekolada': 1.0}
product_cost = {'Chleb_domowy_0_5kg': 0.8, 'Chleb_domowy_0_9kg': 0.9, 'Chleb_na_zakwasie': 0.8, 'Bulki': 0.2, 'Bagietki': 0.2, 'Rogaliki_z_czekolada': 0.2}

# Time available
shifts_per_day = 2
hours_per_shift = 10
hours_per_day = shifts_per_day * hours_per_shift

max_batches = 1
large_number = 10000

sub_production_times = {
    'Ciasto_na_chleb_domowy_0_5kg': {'Dzieza': 4.7, 'Wazenie': 0.29, 'Dzielarka': 0.0, 'Tabula': 0.7, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0},
    'Ciasto_na_chleb_domowy_0_9kg': {'Dzieza': 5, 'Wazenie': 0.2, 'Dzielarka': 0.0, 'Tabula': 0.55, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0},
    'Ciasto_na_chleb_na_zakwasie': {'Dzieza': 8.6, 'Wazenie': 0.22, 'Dzielarka': 0.0, 'Tabula': 0.6, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0},
    'Ciasto_na_bulki': {'Dzieza': 1, 'Wazenie': 0.0, 'Dzielarka': 0.23, 'Tabula': 0.34, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0},
    'Ciasto_na_bagietki': {'Dzieza': 0.95, 'Wazenie': 0.0, 'Dzielarka': 0.24, 'Tabula': 0.0, 'Rogalikarka': 0.28, 'Nadziewarka': 0.0},
    'Ciasto_na_rogaliki_z_czekolada': {'Dzieza': 0.55, 'Wazenie': 0.0, 'Dzielarka': 0.28, 'Tabula': 0.0, 'Rogalikarka': 0.31, 'Nadziewarka': 0.4}
}

production_times = {
    'Chleb_domowy_0_5kg': {'Garownia': 1.0, 'Piec': 0.68, 'Studzenie': 1.8, 'Pakowanie': 0.56},
    'Chleb_domowy_0_9kg': {'Garownia': 1.5, 'Piec': 1.2, 'Studzenie': 2.5, 'Pakowanie': 0.57},
    'Chleb_na_zakwasie': {'Garownia': 1.4, 'Piec': 1.0, 'Studzenie': 2.3, 'Pakowanie': 0.6},
    'Bulki': {'Garownia': 0.38, 'Piec': 0.25, 'Studzenie': 0.48, 'Pakowanie': 0.0},
    'Bagietki': {'Garownia': 0.33, 'Piec': 0.23, 'Studzenie': 0.42, 'Pakowanie': 0.0},
    'Rogaliki_z_czekolada': {'Garownia': 0.25, 'Piec': 0.18, 'Studzenie': 0.82, 'Pakowanie': 0.42}
}


batch_size = {
    "Ciasto_na_chleb_domowy_0_5kg": 240,
    "Ciasto_na_chleb_domowy_0_9kg": 130,
    "Ciasto_na_chleb_na_zakwasie": 200,
    "Ciasto_na_bulki": 200,
    "Ciasto_na_bagietki": 200,
    "Ciasto_na_rogaliki_z_czekolada": 200,
    "Chleb_domowy_0_5kg": 240,
    "Chleb_domowy_0_9kg": 130,
    "Chleb_na_zakwasie": 200,
    "Bulki": 200,
    "Bagietki": 200,
    "Rogaliki_z_czekolada": 200,
}

market_limits = {
    "Chleb_domowy_0_5kg": {"Pon": 100, "Wt": 100, "Sr": 240, "Czw": 420, "Pt": 530, "Sob": 680},
    "Chleb_domowy_0_9kg": {"Pon": 100, "Wt": 100, "Sr": 380, "Czw": 150, "Pt": 250, "Sob": 580},
    "Chleb_na_zakwasie": {"Pon": 100, "Wt": 100, "Sr": 320, "Czw": 180, "Pt": 280, "Sob": 418},
    "Bulki": {"Pon": 100, "Wt": 100, "Sr": 100, "Czw": 530, "Pt": 650, "Sob": 580},
    "Bagietki": {"Pon": 100, "Wt": 100, "Sr": 100, "Czw": 350, "Pt": 650, "Sob": 680},
    "Rogaliki_z_czekolada": {"Pon": 100, "Wt": 100, "Sr": 240, "Czw": 310, "Pt": 400, "Sob": 280},
}

# Machines cost
fridge_cost_per_day = 0.281 * 24 * 1.4 * 0.2 # kW * 24h * price per hour * factor
furnace_cost_per_hour = 82.4 # https://giko.pl/kalkulator-zuzycia-energii-dla-piecow-piekarniczych/


fridge_count = 10
fridge_capacity = 160
end_stock = 0
storage_capacity = fridge_count * fridge_capacity

# Variables for production, sales, and storage of products each day
sub_production_vars = pl.LpVariable.dicts("Subproduct production", ((subproduct, day) for subproduct in subproducts for day in days), lowBound=0, cat='Integer')
production_vars = pl.LpVariable.dicts("Production", ((product, day) for product in products for day in days), lowBound=0, cat='Integer')
# batch_count_vars = pl.LpVariable.dicts("Batches Count", ((product, day) for product in products for day in days), lowBound=0, cat='Integer')
# subbatch_count_vars = pl.LpVariable.dicts("Subproduct Batches Count", ((subproduct, day) for subproduct in subproducts for day in days), lowBound=0, cat='Integer')
product_batch_start_time = pl.LpVariable.dicts("ProductStartTime",  [(product, machine, day) for product in products for machine in machines for day in days], lowBound=0, cat="Continuous")
subproduct_batch_start_time = pl.LpVariable.dicts("Subproduction start time for batch",  [(subproduct, machine, day) for subproduct in subproducts for machine in sub_machines for day in days], lowBound=0, cat="Continuous")
sales_vars = pl.LpVariable.dicts("Sales", ((product, day) for product in products for day in days), lowBound=0, cat='Integer')
# specific_machine_used = pl.LpVariable.dicts("SpecificMachineUsed", [(machine, idx) for (machine, idx) in specific_machines], cat="Binary")
# subbatch_produced = pl.LpVariable.dicts("BatchProducedSubproduct", [(subproduct, day, batch) for subproduct in subproducts for day in days for batch in range(1, max_batches + 1)], cat="Binary")
subbatch_produced = pl.LpVariable.dicts("BatchProducedSubproduct", [(subproduct, day) for subproduct in subproducts for day in days], cat="Binary")
batch_produced = pl.LpVariable.dicts("BatchProduced", [(product, day) for product in products for day in days], cat="Binary")
specific_sub_machine_used = pl.LpVariable.dicts("SpecificMachineUsed", [(machine, subproduct, day) for machine in sub_machines for subproduct in subproducts for day in days], cat="Binary")
specific_machine_used = pl.LpVariable.dicts("SpecificMachineUsed", [(machine, product, day) for machine in machines for product in products for day in days], cat="Binary")
storage_vars = pl.LpVariable.dicts("Storage", ((subproduct, day) for subproduct in subproducts for day in days), lowBound=0, upBound=storage_capacity, cat='Integer')
fridge_count_usage_vars = pl.LpVariable.dicts("FridgeCount", ((day) for day in days), lowBound=0, upBound=fridge_count, cat='Integer')


# xij = pl.LpVariable.dicts("xij", [(machine, idx, i, j) for (machine, idx) in specific_machines for i in subproducts for j in subproducts if i != j], 0, 1, pl.LpBinary)
is_produced_sub = pl.LpVariable.dicts("is_produced",
                                     ((subproduct, machine, day) for subproduct in subproducts
                                                                      for machine in sub_machines
                                                                      for day in days),
                                     cat="Binary")
is_produced = pl.LpVariable.dicts("is_produced",
                                     ((product, machine, day) for product in products
                                                                      for machine in machines
                                                                      for day in days),
                                     cat="Binary")

for subproduct in subproducts:
    for day in days:
        for i, machine in enumerate(sub_machines[1:], start=1):  # Od ważenia do nadziewarki
            prev = 1
            prev_machine = sub_machines[i - prev]

            while sub_production_times[subproduct][prev_machine] == 0.0 or prev_machine == machine:
                prev += 1
                prev_machine = sub_machines[i - prev]

            model += subproduct_batch_start_time[subproduct, machine, day] >= (
                subproduct_batch_start_time[subproduct, prev_machine, day] + sub_production_times[subproduct][prev_machine]
            )

            model += subproduct_batch_start_time[subproduct, machine, day] <= specific_sub_machine_used[machine, subproduct, day] * hours_per_day

        last_machine = sub_machines[-1]
        model += hours_per_day >= (
            subproduct_batch_start_time[subproduct, last_machine, day] + sub_production_times[subproduct][last_machine]
        )

for product in products:
    for day in days:
        for i, machine in enumerate(machines[1:], start=1):  # Od ważenia do nadziewarki
            prev = 1
            prev_machine = machines[i - prev]

            while production_times[product][prev_machine] == 0.0 or prev_machine == machine:
                prev += 1
                prev_machine = machines[i - prev]

            model += product_batch_start_time[product, machine, day] >= (
                product_batch_start_time[product, prev_machine, day] + production_times[product][prev_machine]
            )

            model += product_batch_start_time[product, machine, day] <= specific_machine_used[machine, product, day] * hours_per_day

        last_machine = machines[-1]
        model += hours_per_day >= (
            product_batch_start_time[product, last_machine, day] + production_times[product][last_machine]
        )
'''
        model += subbatch_produced[subproduct, day] * large_number >= subproduct_batch_start_time[subproduct, last_machine, last_idx, day]

        # Brak nakładania się produkcji na maszynie
        for machine, idx in specific_machines[:6]:  # Zakres maszyn
            for subproduct1 in subproducts:
                for subproduct2 in subproducts:
                    if subproduct1 != subproduct2:
                        model += (subproduct_batch_start_time[subproduct1, machine, idx, day]
                                  >= subproduct_batch_start_time[subproduct2, machine, idx, day] + production_times[subproduct2][machine]) | \
                                 (subproduct_batch_start_time[subproduct2, machine, idx, day]
                                  >= subproduct_batch_start_time[subproduct1, machine, idx, day] + production_times[subproduct1][machine])

                        # model += specific_machine_used[machine, idx, subproduct1, day] + specific_machine_used[machine, idx, subproduct2, day] <= 1
'''
# Ograniczenie: produkcja subproduktu tylko na maszynach, gdzie czas produkcji > 0
for subproduct in subproducts:
    for machine in sub_machines:
        for day in days:
            if sub_production_times[subproduct][machine] == 0:
                model += is_produced_sub[subproduct, machine, day] == 0

# Ograniczenie: produkcja produktu tylko na maszynach, gdzie czas produkcji > 0
for product in products:
    for machine in machines:
        for day in days:
            if production_times[product][machine] == 0:
                model += is_produced[product, machine, day] == 0
                

# Warunek: jeśli subprodukt jest produkowany, to na wszystkich maszynach (z czasem > 0)
for subproduct in subproducts:
    machines_with_production = [machine for machine in sub_machines if sub_production_times[subproduct][machine] > 0]
    for day in days:
        if machines_with_production:
            # Ograniczenie wymuszające produkcję na wszystkich maszynach albo na żadnej
            for machine in machines_with_production:
                    model += pl.lpSum(is_produced_sub[subproduct, m, day] for m in machines_with_production) == \
                            len(machines_with_production) * is_produced_sub[subproduct, machine, day]
        
        # pozowolenie subbatch_produced na wartość 1 jeżeli na maszyny działały dla danego wyrobu
        model += subbatch_produced[subproduct, day] <= pl.lpSum(is_produced_sub[subproduct, machine, day] for machine in machines_with_production)

# Warunek: jeśli produkt jest produkowany, to na wszystkich maszynach (z czasem > 0)
for product in products:
    machines_with_production = [machine for machine in machines if production_times[product][machine] > 0]
    for day in days:
        if machines_with_production:
            # Ograniczenie wymuszające produkcję na wszystkich maszynach albo na żadnej
            for machine in machines_with_production:
                    model += pl.lpSum(is_produced[product, m, day] for m in machines_with_production) == \
                            len(machines_with_production) * is_produced[product, machine, day]
        
        # pozowolenie subbatch_produced na wartość 1 jeżeli na maszyny działały dla danego wyrobu
        model += batch_produced[product, day] <= pl.lpSum(is_produced[product, machine, day] for machine in machines_with_production)

# Aktualizacja ograniczenia: brak nakładania się produkcji
big_M = 1000000
for subproduct1 in subproducts:
    for subproduct2 in subproducts:
        if subproduct1 != subproduct2:
            machines_with_production = [machine for machine in sub_machines if sub_production_times[subproduct1][machine] > 0 and sub_production_times[subproduct2][machine] > 0]
            for machine in machines_with_production: 
                for day in days:
                    # Dodanie binarnych zmiennych pomocniczych
                    y1 = pl.LpVariable(f"y1_{subproduct1}_{subproduct2}_{machine}_{day}", cat="Binary")
                    y2 = pl.LpVariable(f"y2_{subproduct1}_{subproduct2}_{machine}_{day}", cat="Binary")
                    y3 = pl.LpVariable(f"y3_{subproduct1}_{subproduct2}_{machine}_{day}", cat="Binary")
                    y4 = pl.LpVariable(f"y4_{subproduct1}_{subproduct2}_{machine}_{day}", cat="Binary")

                    # Powiązanie warunków logicznych z binarnymi zmiennymi
                    model += subproduct_batch_start_time[subproduct1, machine, day] >= \
                                subproduct_batch_start_time[subproduct2, machine, day] + sub_production_times[subproduct2][machine] - \
                                big_M * (1 - y1)
                    
                    model += subproduct_batch_start_time[subproduct2, machine, day] >= \
                                subproduct_batch_start_time[subproduct1, machine, day] + sub_production_times[subproduct1][machine] - \
                                big_M * (1 - y2)

                    model += is_produced_sub[subproduct1, machine, day] + y3 == 1
                    model += is_produced_sub[subproduct2, machine, day] + y4 == 1

                    # Zapewnienie, że przynajmniej jedno ograniczenie jest spełnione
                    model += y1 + y2 + y3 + y4 == 1

for product1 in products:
    for product2 in products:
        if product1 != product2:
            machines_with_production = [machine for machine in machines if production_times[product1][machine] > 0 and production_times[product2][machine] > 0]
            for machine in machines_with_production: 
                for day in days:
                    # Dodanie binarnych zmiennych pomocniczych
                    y1 = pl.LpVariable(f"y1_{product1}_{product2}_{machine}_{day}", cat="Binary")
                    y2 = pl.LpVariable(f"y2_{product1}_{product2}_{machine}_{day}", cat="Binary")
                    y3 = pl.LpVariable(f"y3_{product1}_{product2}_{machine}_{day}", cat="Binary")
                    y4 = pl.LpVariable(f"y4_{product1}_{product2}_{machine}_{day}", cat="Binary")

                    # Powiązanie warunków logicznych z binarnymi zmiennymi
                    model += product_batch_start_time[product1, machine, day] >= \
                                product_batch_start_time[product2, machine, day] + production_times[product2][machine] - \
                                big_M * (1 - y1)
                    
                    model += product_batch_start_time[product2, machine, day] >= \
                                product_batch_start_time[product1, machine, day] + production_times[product1][machine] - \
                                big_M * (1 - y2)

                    model += is_produced[product1, machine, day] + y3 == 1
                    model += is_produced[product2, machine, day] + y4 == 1

                    # Zapewnienie, że przynajmniej jedno ograniczenie jest spełnione
                    model += y1 + y2 + y3 + y4 == 1


for subproduct, product in zip(subproducts, products):
    model += storage_vars[subproduct, 'Pon'] == sub_production_vars[subproduct, 'Pon'] - production_vars[product, 'Pon']
    for i in range(1, len(days)):
        day = days[i]
        prev_day = days[i-1]
        model += storage_vars[subproduct, day] == storage_vars[subproduct, prev_day] + sub_production_vars[subproduct, day] - production_vars[product, day]

for day in days:
    model += storage_capacity >= sum(storage_vars[subproduct, day] for subproduct in subproducts)
    model += fridge_count_usage_vars[day] * fridge_capacity >= sum(storage_vars[subproduct, day] for subproduct in subproducts)

for product, subproduct in zip(products, subproducts):
    for day in days:
        model += batch_produced[product, day] * batch_size[product] >= production_vars[product, day]
        model += subbatch_produced[subproduct, day] * batch_size[subproduct] >= sub_production_vars[subproduct, day]
        # model += sub_production_vars[subproduct, day] >= production_vars[product, day]
        model += sales_vars[product, day] == production_vars[product, day]
    model += sales_vars[product, 'Sr'] >= 0.01*market_limits[product]['Wt'], f"Max_sales_{product}_{'Sr'}"


# Objective function: Maximize profit from sales minus costs
model += pl.lpSum([sales_vars[product, day] * product_income[product] for product in products for day in days]) \
                    - pl.lpSum([production_vars[product, day] * product_cost[product] for product in products for day in days]) \
                    - pl.lpSum([sub_production_vars[subproduct, day] * sub_product_cost[subproduct] for subproduct in subproducts for day in days])


solver = pl.PULP_CBC_CMD(timeLimit=600)

model.solve(solver)

# model.solve()

# Results
print("Status:", pl.LpStatus[model.status])
if pl.LpStatus[model.status] == 'Optimal':
    print("Total Profit:", pl.value(model.objective))

    print(f"{'Product':<10} {'day':<10} {'SubProduction':<10} {'Production':<12} {'Sales':<10} {'Storage':<10}")

    for product, subproduct in zip(products, subproducts):
        for day in days:
            subproduction = sub_production_vars[subproduct, day].varValue
            production = production_vars[product, day].varValue
            sales = sales_vars[product, day].varValue
            storage = storage_vars[subproduct, day].varValue

            print(f"{product:<10} {day:<10} {subproduction:<12} {production:<12} {sales:<10} {storage:<10}")

    # print("\nProduction Time Usage per Product and Day:")
    # for day in days:
    #     print(f"\Day: {day}")
    #     for product in products:
    #         total_production_time = sum(
    #             batch_count_vars[product, day].varValue * production_times[product]s.get(machine, 0) 
    #             for machine in machines #.keys()
    #         )
    #         print(f"  Product: {product}, Total Production Time: {total_production_time:.2f} hours")

else:
    print("No optimal solution was found.")
    # Analiza ograniczeń
    for name, constraint in model.constraints.items():
        if constraint.pi < 0:
            print(f"Ograniczenie {name}: {constraint}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gromiusz/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/1c95c53dc7cf45e98a93188e4839f9ce-pulp.mps -max -sec 600 -timeMode elapsed -branch -printingOptions all -solution /tmp/1c95c53dc7cf45e98a93188e4839f9ce-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2999 COLUMNS
At line 15828 RHS
At line 18823 BOUNDS
At line 21155 ENDATA
Problem MODEL has 2994 rows, 2511 columns and 8112 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 600
Option for timeMode changed from cpu to elapsed
Continuous objective value is 7416 - 0.01 seconds
Cgl0002I 57 variables fixed
Cgl0003I 22 fixed, 0 tightened bounds, 984 strengthened rows, 276 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 759 strengthened rows, 108 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 803 strengthened rows, 6 substitutions
Cgl0003I 0 fixed, 0

In [44]:
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)  # Ustaw brak limitu szerokości kolumny


# Po rozwiązaniu problemu upewnijmy się, że rozwiązanie jest optymalne
if pl.LpStatus[model.status] != 'Optimal':
    print("Model did not solve to optimality.")
    exit()

# Pobierz wartości czasu rozpoczęcia
start_times = {
    (subproduct, machine, day): subproduct_batch_start_time[subproduct, machine, day].varValue
    for subproduct in subproducts for machine in sub_machines for day in days
    if subproduct_batch_start_time[subproduct, machine, day].varValue is not None
}

# Oblicz czas zakończenia
end_times = {
    (subproduct, machine, day): start_times[subproduct, machine, day] + sub_production_times[subproduct][machine]
    for subproduct, machine, day in start_times
}

# Wersja 1: Widok z perspektywy subproduktu
subproduct_schedule = []
for subproduct in subproducts:
    for day in days:
        for machine in sub_machines:
            if (subproduct, machine, day) in start_times:
                start = start_times[(subproduct, machine, day)]
                end = end_times[(subproduct, machine, day)]
                subproduct_schedule.append({
                    'Subproduct': subproduct,
                    'Day': day,
                    'Machine': machine,
                    'Start Time': start,
                    'End Time': end
                })

subproduct_df = pd.DataFrame(subproduct_schedule)
print("\nSubproduct Schedule:\n")
print(subproduct_df.sort_values(['Day', 'Subproduct', 'Start Time']))

# Wersja 2: Oś czasu dla każdej maszyny
# Tworzenie mapowania subproduktów na litery
subproduct_mapping = {subproduct: chr(65 + i) for i, subproduct in enumerate(subproducts)}

# Długość jednego znaku odpowiadająca 15 minutom
time_slot_length = 0.2  # 15 minut w godzinach
slots_per_day = int(hours_per_day / time_slot_length)  # Liczba slotów w ciągu dnia

def visualize_timeline(machine, day):
    timeline = [None for _ in range(slots_per_day)]
    for subproduct in subproducts:
        if (subproduct, machine, day) in start_times:
            start = start_times[(subproduct, machine, day)]
            end = end_times[(subproduct, machine, day)]
            start_slot = int(start / time_slot_length)
            end_slot = int(end / time_slot_length)
            for i in range(start_slot, end_slot):
                if timeline[i] is None:
                    timeline[i] = subproduct_mapping[subproduct]  # Wstaw literę dla subproduktu
                else:
                    # Rozstrzyganie konfliktów (krótsze zadanie)
                    prev_subproduct = timeline[i]
                    prev_duration = (
                        end_times[(subproducts[ord(prev_subproduct) - 65], machine, day)] -
                        start_times[(subproducts[ord(prev_subproduct) - 65], machine, day)]
                    )
                    curr_duration = end - start
                    if curr_duration < prev_duration:
                        timeline[i] = subproduct_mapping[subproduct]

    # Zamień None na "_" i pozostałe na stringi
    return "".join(char if char is not None else "_" for char in timeline)

machine_timeline = []

for day in days:
    for machine in sub_machines:
        timeline = visualize_timeline(machine, day)
        machine_timeline.append({
            'Machine': machine,
            'Day': day,
            'Timeline': timeline
        })

machine_timeline_df = pd.DataFrame(machine_timeline)
print("\nMachine Usage Timeline:\n")
print(machine_timeline_df)

# Opcjonalnie: zapis wyników do pliku CSV
subproduct_df.to_csv("subproduct_schedule.csv", index=False)
machine_timeline_df.to_csv("machine_timeline.csv", index=False)
# Wyświetlenie legendy
print("\nLegenda Subproduktów:")
for subproduct, letter in subproduct_mapping.items():
    print(f"{letter}: {subproduct}")




Subproduct Schedule:

                         Subproduct  Day      Machine  Start Time  End Time
72               Ciasto_na_bagietki  Pon       Dzieza        0.00      0.95
74               Ciasto_na_bagietki  Pon    Dzielarka       19.48     19.72
76               Ciasto_na_bagietki  Pon  Rogalikarka       19.72     20.00
73               Ciasto_na_bagietki  Pon      Wazenie       20.00     20.00
75               Ciasto_na_bagietki  Pon       Tabula       20.00     20.00
..                              ...  ...          ...         ...       ...
98   Ciasto_na_rogaliki_z_czekolada   Wt    Dzielarka       19.01     19.29
100  Ciasto_na_rogaliki_z_czekolada   Wt  Rogalikarka       19.29     19.60
101  Ciasto_na_rogaliki_z_czekolada   Wt  Nadziewarka       19.60     20.00
97   Ciasto_na_rogaliki_z_czekolada   Wt      Wazenie       20.00     20.00
99   Ciasto_na_rogaliki_z_czekolada   Wt       Tabula       20.00     20.00

[108 rows x 5 columns]

Machine Usage Timeline:

        Machine

In [45]:
# Pobierz wartości czasu rozpoczęcia
start_times = {
    (product, machine, day): product_batch_start_time[product, machine, day].varValue
    for product in products for machine in machines for day in days
    if product_batch_start_time[product, machine, day].varValue is not None
}

# Oblicz czas zakończenia
end_times = {
    (product, machine, day): start_times[product, machine, day] + production_times[product][machine]
    for product, machine, day in start_times
}

# Wersja 1: Widok z perspektywy produktu
product_schedule = []
for product in products:
    for day in days:
        for machine in machines:
            if (product, machine, day) in start_times:
                start = start_times[(product, machine, day)]
                end = end_times[(product, machine, day)]
                product_schedule.append({
                    'Product': product,
                    'Day': day,
                    'Machine': machine,
                    'Start Time': start,
                    'End Time': end
                })

product_df = pd.DataFrame(product_schedule)
print("\nProduct Schedule:\n")
print(product_df.sort_values(['Day', 'Product', 'Start Time']))

# Wersja 2: Oś czasu dla każdej maszyny
# Tworzenie mapowania produktów na litery
product_mapping = {product: chr(65 + i) for i, product in enumerate(products)}

# Długość jednego znaku odpowiadająca 15 minutom
time_slot_length = 0.2  # 15 minut w godzinach
slots_per_day = int(hours_per_day / time_slot_length)  # Liczba slotów w ciągu dnia

def visualize_timeline(machine, day):
    timeline = [None for _ in range(slots_per_day)]
    for product in products:
        if (product, machine, day) in start_times:
            start = start_times[(product, machine, day)]
            end = end_times[(product, machine, day)]
            start_slot = int(start / time_slot_length)
            end_slot = int(end / time_slot_length)
            for i in range(start_slot, end_slot):
                if timeline[i] is None:
                    timeline[i] = product_mapping[product]  # Wstaw literę dla produktu
                else:
                    # Rozstrzyganie konfliktów (krótsze zadanie)
                    prev_product = timeline[i]
                    prev_duration = (
                        end_times[(products[ord(prev_product) - 65], machine, day)] -
                        start_times[(products[ord(prev_product) - 65], machine, day)]
                    )
                    curr_duration = end - start
                    if curr_duration < prev_duration:
                        timeline[i] = product_mapping[product]

    # Zamień None na "_" i pozostałe na stringi
    return "".join(char if char is not None else "_" for char in timeline)

machine_timeline = []

for day in days:
    for machine in machines:
        timeline = visualize_timeline(machine, day)
        machine_timeline.append({
            'Machine': machine,
            'Day': day,
            'Timeline': timeline
        })

machine_timeline_df = pd.DataFrame(machine_timeline)
print("\nMachine Usage Timeline:\n")
print(machine_timeline_df)

# Opcjonalnie: zapis wyników do pliku CSV
product_df.to_csv("product_schedule.csv", index=False)
machine_timeline_df.to_csv("machine_timeline.csv", index=False)
# Wyświetlenie legendy
print("\nLegenda Produktów:")
for product, letter in product_mapping.items():
    print(f"{letter}: {product}")



Product Schedule:

                 Product  Day    Machine  Start Time  End Time
48              Bagietki  Pon   Garownia        1.78      2.11
49              Bagietki  Pon       Piec       11.00     11.23
50              Bagietki  Pon  Studzenie       11.81     12.23
51              Bagietki  Pon  Pakowanie       20.00     20.00
36                 Bulki  Pon   Garownia        0.00      0.38
..                   ...  ...        ...         ...       ...
31     Chleb_na_zakwasie   Wt  Pakowanie       19.40     20.00
64  Rogaliki_z_czekolada   Wt   Garownia        0.00      0.25
65  Rogaliki_z_czekolada   Wt       Piec        9.07      9.25
66  Rogaliki_z_czekolada   Wt  Studzenie       15.51     16.33
67  Rogaliki_z_czekolada   Wt  Pakowanie       17.85     18.27

[72 rows x 5 columns]

Machine Usage Timeline:

      Machine  Day  \
0    Garownia  Pon   
1        Piec  Pon   
2   Studzenie  Pon   
3   Pakowanie  Pon   
4    Garownia   Wt   
5        Piec   Wt   
6   Studzenie   Wt   